# Homework #2: Identifying distinctive words

We're going to explore some basic forms of text analysis, using David Robinson's dataset of tweets made from the account of Donald J. Trump.

To begin, let's import some modules we're going to need later, and also read in the Trump data.

In [4]:
import os, csv, math, re
import pandas as pd
import numpy as np

from collections import Counter
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer

from pathlib import Path

## Different ways of identifying "distinctive" words

In this section we'll explore Dunning's log-likelihood, and also think about the strengths and weaknesses of "distinctive" words as evidence.

First let's glance at the Trump dataset.

In [16]:
relativepath = Path('../../data/trump.csv')
trump = pd.read_csv(relativepath, encoding = 'latin1')
print('We have ', trump.shape[0], 'rows, and')
print(trump.shape[1], 'columns.')
trump.head()

We have  1512 rows, and
18 columns.


,Unnamed: 0,Unnamed: 0.1,text,favorited,favoriteCount,replyToSN,created,truncated,replyToSID,id,replyToUID,statusSource,screenName,retweetCount,isRetweet,retweeted,longitude,latitude
0,0,1,My economic policy speech will be carried live...,False,9214,NaN,2016-08-08 15:20:44,False,NaN,762669882571980801,NaN,"<a href=""http://twitter.com/download/android"" ...",realDonaldTrump,3107,False,False,NaN,NaN
1,1,2,"Join me in Fayetteville, North Carolina tomorr...",False,6981,NaN,2016-08-08 13:28:20,False,NaN,762641595439190016,NaN,"<a href=""http://twitter.com/download/iphone"" r...",realDonaldTrump,2390,False,False,NaN,NaN
2,2,3,"#ICYMI: ""Will Media Apologize to Trump?"" https...",False,15724,NaN,2016-08-08 00:05:54,False,NaN,762439658911338496,NaN,"<a href=""http://twitter.com/download/iphone"" r...",realDonaldTrump,6691,False,False,NaN,NaN
3,3,4,"Michael Morell, the lightweight former Acting ...",False,19837,NaN,2016-08-07 23:09:08,False,NaN,762425371874557952,NaN,"<a href=""http://twitter.com/download/android"" ...",realDonaldTrump,6402,False,False,NaN,NaN
4,4,5,The media is going crazy. They totally distort...,False,34051,NaN,2016-08-07 21:31:46,False,NaN,762400869858115588,NaN,"<a href=""http://twitter.com/download/android"" ...",realDonaldTrump,11717,False,False,NaN,NaN


#### Basic functions

For a lot of the work we do today, we're going to want to construct dictionaries that hold the frequencies of words in different categories: poetry or fiction, Trump-iphone or Trump-android. To do this we'll need to break text into words, count the words in each text, and then add up the counts by category.

Let's define some functions that do this. (You can find more polished versions of these functions in the ```nltk``` module.)


In [10]:
def split_into_words(any_chunk_of_text):
    lowercase_text = any_chunk_of_text.lower()
    split_words = re.split("\W+", lowercase_text)
    return split_words

def create_vocab(seq_of_strings, n):
    ''' Given a sequence of text snippets, this function
    returns the n most common words. We'll use this to
    create a limited 'vocabulary'.
    '''
    vocab = Counter()
    
    for astring in seq_of_strings:
        thesewords = [x for x in split_into_words(astring) if len(x) > 0]
        these_counts = Counter(thesewords)
        vocab = vocab + these_counts
        
    topn = [x[0] for x in vocab.most_common(n)]
    return topn

# Let's test the vocabulary function.
vocab = create_vocab(trump['text'], 4000)
print(vocab[0:10])    

['the', 't', 'co', 'https', 'to', 'and', 'a', 'i', 'in', 'is']


#### A few more basic functions

Once we have a vocabulary, we're going to want to divide our texts into categories, create Counters summing the word frequencies in those categories, and then compare the two Counters to find words that are overrepresented in one category relative to the other.

There are several ways we could define "overrepresented." We'll use Robinson's simple log-odds measure, as well as Dunning's log-likelihood.

In [14]:
def logodds(countsA, countsB, word):
    ''' Straightforward.
    '''
    
    odds = (countsA[word] + 1) / (countsB[word] + 1)
    
    # Why do we add 1 on both sides? Two reasons. The hacky one is 
    # that otherwise we'll get a division-by-zero error whenever
    # word isn't present in countsB. The more principled reason
    # is that this technique (called Laplacian smoothing) tends
    # to reduce the dramatic disproportion likely to be found in
    # very rare words.
    
    return math.log(odds)

def signed_dunnings(countsA, totalA, countsB, totalB, word):
    ''' Less straightforward. This function calculates a signed (+1 / -1)
    version of Dunning's log likelihood. Intuitively, this is a number 
    that gets larger as the frequency of the word in our two corpora
    diverges from its EXPECTED frequency -- i.e., the frequency it would
    have if it were equally distributed over both. But it also tends to get
    larger as the raw frequency of the word increases.
    
    Note that this function requires two additional arguments:
    the total number of words in A and B. We could calculate that inside
    the function, but it's faster to calculate it just once, outside the function.
    
    Also note: the strict definition of Dunnings has no 'sign': it gets bigger
    whether a word is overrepresented in A or B. I've edited that so that Dunnings
    is positive if overrepresented in A, and negative if overrepresented in B.
    '''
    if word not in countsA and word not in countsB:
        return 0
    
    # the raw frequencies of this word in our two corpora
    # still doing a little Laplacian smoothing here
    a = countsA[word] + 0.1
    b = countsB[word] + 0.1
    
    # now let's calculate the expected number of times this
    # word would occur in both if the frequency were constant
    # across both
    overallfreq = (a + b) / (totalA + totalB)
    expectedA = totalA * overallfreq
    expectedB = totalB * overallfreq
    
    # and now the Dunning's formula
    dunning = 2 * ((a * math.log(a / expectedA)) + (b * math.log(b / expectedB)))
    
    if a < expectedA:
        return -dunning
    else:   
        return dunning

# a set of common words is often useful
stopwords = {'a', 'an', 'are', 'and', 'but', 'or', 'that', 'this', 'so', 
             'all', 'at', 'if', 'in', 'i', 'is', 'was', 'by', 'of', 'to', 
             'the', 'be', 'you', 'were'}

# finally, one more function: given a list of tuples like
testlist = [(10, 'ten'), (2000, 'two thousand'), (0, 'zero'), (-1, 'neg one'), (8, 'eight')]
# we're going to want to sort them and print the top n and bottom n

def headandtail(tuplelist, n):
    tuplelist.sort(reverse = True)
    print("TOP VALUES:")
    for i in range(n):
        print(tuplelist[i][1], tuplelist[i][0])
    
    print()
    print("BOTTOM VALUES:")
    lastindex = len(tuplelist) - 1
    for i in range(lastindex, lastindex - n, -1):
        print(tuplelist[i][1], tuplelist[i][0])
        
headandtail(testlist, 2)
    

TOP VALUES:
two thousand 2000
ten 10

BOTTOM VALUES:
neg one -1
zero 0


## Exercise 1: Is Dunning's a better measure than logodds for Trump's tweets?

Let's put all these functions together to answer that question.

I've sketched the outline of a program below in "pseudocode," which
describes what needs to be done. Translate that into real Python code, using
the functions defined above. First use Robinson's logodds function and try to
replicate his results. See what happens if you do (or don't) remove stopwords
and tweets that begin with a quote.
                                                   
Then edit your code to use Dunning's log likelihood. Does that seem to be a better (more revealing) measure of overrepresentation? How would we decide?

In [20]:
vocab = create_vocab(trump['text'], 5000)
vocab = list(set(vocab) - stopwords)

numrows = trump.shape[0]
android = Counter()
iphone = Counter()

for i in range(numrows):
        
    counts = Counter(split_into_words(trump['text'][i]))
    if 'iphone' in trump['statusSource'][i]:
        iphone = iphone + counts
    elif 'android' in trump['statusSource'][i]:
        android = android + counts

iphonesum = sum(iphone.values())
androidsum = sum(android.values())

tuplelist = []

for word in vocab:
    # g = signed_dunnings(iphone, iphonesum, android, androidsum, word)
    g = logodds(iphone, android, word)
    tuplelist.append((g, word))

headandtail(tuplelist, 25)

TOP VALUES:
imwithyou 3.044522437723423
trumppence16 2.70805020110221
join 2.662587827025453
americafirst 2.6390573296152584
votetrump 2.4849066497880004
video 2.4849066497880004
7pm 2.4849066497880004
trump2016 2.439444275711243
rncincle 2.3978952727983707
inprimary 2.3978952727983707
https 2.221013439300234
co 2.2044841373490236
wiprimary 2.1972245773362196
officers 2.1972245773362196
fitn 2.1972245773362196
crookedhillary 2.1972245773362196
3pm 2.0794415416798357
tickets 2.0149030205422647
votetrumpnh 1.9459101490553132
refugees 1.9459101490553132
gopconvention 1.9459101490553132
tomorrow 1.791759469228055
greatly 1.791759469228055
gopdebate 1.791759469228055
daytona 1.791759469228055

BOTTOM VALUES:
realdonaldtrump -4.143134726391533
p -3.1780538303479458
00 -2.9444389791664407
v -2.890371757896165
â¼ã -2.772588722239781
two -2.70805020110221
badly -2.639057329615259
crazy -2.639057329615259
megynkelly -2.639057329615259
weak -2.5649493574615367
re -2.4849066497880004
spent -2.4849

## Exercise 2: Apply the same methods to a more literary dataset.

I've also provided a dataset of roughly 1026 snippets from nineteenth-century poetry and fiction. The code below should read it in. Run that, then copy and paste the code you worked up for Trump, and edit it so it provides the most distinctive words for poetry (versus fiction).

If we have time, it may also be worth distinguishing poetry reviewed in elite journals from poetry that wasn't.


In [9]:
relativepath = os.path.join('..', 'data', 'weekfour', 'poefic.csv')
poefic = pd.read_csv(relativepath)
poefic.head()

,date,author,title,genre,reception,text
0,1908,"Robins, Elizabeth,",The convert,fiction,elite,"looked like decent artisans, but more who bore..."
1,1871,"Lytton, Edward Bulwer Lytton,",The coming race,fiction,elite,"called the "" Easy Time "" (with which what I ma..."
2,1872,"Butler, Samuel,","Erewhon, or, Over the range",fiction,elite,the curtain ; on this I let it drop and retrea...
3,1900,"Barrie, J. M.",Tommy and Grizel,fiction,elite,"at you !"" he said. ""Dear eyes, "" said she. ""Th..."
4,1873,"Ritchie, Anne Thackeray,",Old Kensington,fiction,elite,"furious; I have not dared tell her, poor creat..."


In [10]:
# Code for Exercise 2

# The main thing you will need to change is the code that
# identifies rows as belonging to one of two corpora.


## Using corpora to create a "meaning space."

Contrasting two corpora can be revealing, but sometimes we want to think about the relations between individual texts or words. To do that, we often represent them as vectors in a multi-dimensional space.

The simplest way to do this is to create a DataFrame where rows are documents and columns are word — a document-term matrix. Here's a function that does that. It requires a pre-defined vocabulary (list of words) as well as a list (or numpy vector) of texts.


In [46]:
def doc_term_matrix(vocab, textvector):
    ''' Transform the textvector into a document-term matrix
    with one column for each word in vocab.
    '''
    
    n = len(textvector)
    vocabset = set(vocab)
    # making a set so we can check membership quickly;
    # it's much faster in a set than in a list
    
    termdictionary = dict()
    for word in vocab:
        termdictionary[word] = np.zeros(n)
    for i, text in enumerate(textvector):
        counts = tokenize(text)
        for word, count in counts.items():
            if word in vocabset:
                termdictionary[word][i] += count
    
    dtmatrix = pd.DataFrame(termdictionary, columns = vocab)
    return dtmatrix

# A nice arcane trick to perform on a document-term matrix
# is to squash it into a smaller number of dimensions. This
# often reveals relationships between words that don't
# necessarily, literally occur together. The technique is called
# Latent Semantic Analysis.

def lsa_matrix(dtmatrix, vocab, number_of_dimensions):
    lsa = TruncatedSVD(number_of_dimensions, algorithm = 'arpack')
    dtm_lsa = lsa.fit_transform(dtmatrix)
    dtm_lsa = Normalizer(copy=False).fit_transform(dtm_lsa)
    lsamatrix = pd.DataFrame(lsa.components_, columns = vocab)
    
    return lsamatrix

def cosine_similarity(vector1, vector2):
    dot_product = np.sum(vector1 * vector2)
    # we assume these are numpy vectors and can be
    # multiplied elementwise
    
    magnitude = math.sqrt(sum([val**2 for val in vector1])) * math.sqrt(sum([val**2 for val in vector2]))
    if not magnitude:
        return 0
    else:
        return dot_product/magnitude

## Exercise 3: Finding words that are close in "meaning space."

Following Widdows, we'll measure semantic similarity as the cosine similarity between vectors defined by a word's distribution across documents.

Let's try this both in the space defined by Trump tweets and in the space defined by 19c literature.

In [50]:
vocab = create_vocab(trump['text'], 5000)
dtm = doc_term_matrix(vocab, trump['text'])
lsa = lsa_matrix(dtm, vocab, 25)
dtm.head()

,the,to,and,a,in,is,i,you,of,will,...,windham,nato's,9pm,@stephbewitching,supposedly,https://t.co/2towwtheef,https://t.co/n43cpejiqa,https://t.co/auszsnozlm,among,hitting
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
# Code for exercise 3

# Let's start by getting a vocabulary,
# and a doc-term matrix, as well as a
# squashed (lsa) version of that matrix.

vocab = create_vocab(trump['text'], 3000)
dtm = doc_term_matrix(vocab, trump['text'])
lsa = lsa_matrix(dtm, vocab, 25)

def find_matches(amatrix, word):
    columnA = amatrix[word]
    
    vocab = amatrix.columns.values
    tuplelist = []
    for w in vocab:
        columnB = amatrix[w]
        cosinesim = cosine_similarity(columnA, columnB)
        tuplelist.append((cosinesim, w))
    
    headandtail(tuplelist, 10)



In [63]:
user_word = input('word? ')
find_matches(lsa, user_word) 



word? weak
TOP VALUES:
weak 1.0
openly 0.6872722879003443
losses 0.626152250163958
desperate 0.6224385022881581
team 0.6223334214785835
leadership 0.6176981361352789
act 0.6161027847845036
unfit 0.5817734047796101
https://t.co/3ezg620fpt 0.5691954802090683
figure,wants 0.5563717702230214

BOTTOM VALUES:
ryan -0.5659272121712704
picks -0.5111880328998333
donât -0.49501014875028715
meaningless -0.46681146235839505
went -0.454161210091449
hope -0.45236006444886045
front -0.43537197005938383
page -0.43537197005938383
megan -0.4325629283517442
$1 -0.43243588978981606
